In [1]:
import pinecone
import os 

PINECONE_API_KEY = "bf349a22-3a4e-487a-a05a-14f9193ed5e6"
PINECONE_ENV_KEY = "asia-southeast1-gcp-free"

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV_KEY
)

c:\Users\86183\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
index_name = "pinecone-example"

if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

pinecone.create_index(
    name=index_name,
    dimension=2,
    metric="euclidean",
    shards=1
)

index = pinecone.Index(
    index_name=index_name
)

In [3]:
import pandas as pd

df = pd.DataFrame()
df["id"] = ["A", "B", "C", "D", "E"]
df["vector"] = [[1., 1.], [1., 2.], [3., 4.], [5., 5.], [7., 7.]]

df 

,id,vector
0,A,"[1.0, 1.0]"
1,B,"[1.0, 2.0]"
2,C,"[3.0, 4.0]"
3,D,"[5.0, 5.0]"
4,E,"[7.0, 7.0]"


In [4]:
index.upsert(
    vectors=zip(df["id"], df["vector"])
)

{'upserted_count': 5}

In [5]:
fetch_results = index.fetch(
    ids=["A", "B"]
)
fetch_results

{'namespace': '',
 'vectors': {'A': {'id': 'A', 'values': [1.0, 1.0]},
             'B': {'id': 'B', 'values': [1.0, 2.0]}}}

In [7]:
query_results = index.query(
    queries=[[1.1, 1.1], [2.2, 2.2]],
    top_k=2,
    include_values=True
)
query_results

{'results': [{'matches': [{'id': 'A',
                           'score': 0.0199999809,
                           'values': [1.0, 1.0]},
                          {'id': 'B',
                           'score': 0.819999695,
                           'values': [1.0, 2.0]}],
              'namespace': ''},
             {'matches': [{'id': 'B',
                           'score': 1.47999954,
                           'values': [1.0, 2.0]},
                          {'id': 'A', 'score': 2.88, 'values': [1.0, 1.0]}],
              'namespace': ''}]}

In [8]:
fetch_results = index.fetch(
    ids=["A"]
)
fetch_results

{'namespace': '', 'vectors': {'A': {'id': 'A', 'values': [1.0, 1.0]}}}

In [9]:
index.upsert(
    vectors=[("A", [0., 0.])]
)

{'upserted_count': 1}

In [10]:
fetch_results = index.fetch(
    ids=["A"]
)
fetch_results

{'namespace': '', 'vectors': {'A': {'id': 'A', 'values': [0.0, 0.0]}}}

In [11]:
index.delete(
    ids=["A"]
)

{}

In [12]:
fetch_results = index.fetch(
    ids=["A", "B"]
)
fetch_results

{'namespace': '', 'vectors': {'B': {'id': 'B', 'values': [1.0, 2.0]}}}

In [13]:
index.describe_index_stats()

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4}

In [14]:
pinecone.delete_index(index_name)